In [11]:
import pandas as pd
# Chuẩn bị dữu liệu
# Load các file dữ liệu đã làm sạch
df_ung_vien = pd.read_csv('D:/Nhom_5_Case_6/Data_Clean/cleaned_candidate_profile.csv')
df_diem = pd.read_csv('D:/Nhom_5_Case_6/Data_Clean/cleaned_interview_score.csv')
df_luong = pd.read_csv('D:/Nhom_5_Case_6/Data_Clean/cleaned_salary_offer.csv')
# 1. Chuẩn hóa cột candidate_id để đảm bảo tính nhất quán khi ghép bảng
df_ung_vien['candidate_id'] = df_ung_vien['candidate_id'].str.upper().str.strip()
df_diem['candidate_id'] = df_diem['candidate_id'].str.upper().str.strip()
df_luong['candidate_id'] = df_luong['candidate_id'].str.upper().str.strip()

# 2. Chuẩn hóa Vị trí (Quan trọng để Pivot không bị tách dòng sai)
df_luong['position'] = df_luong['position'].str.strip().str.title()
tu_dien_vi_tri = {
    'Dev': 'Python Developer',
    'Python Dev': 'Python Developer',
    'Data Analyst': 'Data Analyst'
}
df_luong['position'] = df_luong['position'].map(tu_dien_vi_tri).fillna(df_luong['position'])

# 3. Ghép bảng (Merge) tạo dữ liệu tổng hợp
# Cần ghép để có cột Vị trí (từ bảng Lương) và cột Điểm (từ bảng Điểm) nằm chung 1 chỗ
df_master = pd.merge(df_ung_vien, df_diem, on='candidate_id', how='left')
df_master = pd.merge(df_master, df_luong, on='candidate_id', how='left')


# --- BƯỚC 2: THỰC HIỆN PIVOT TABLE & STACK ---

# === YÊU CẦU 1: Pivot Điểm trung bình (Vị trí x Vòng thi) ===
# Mục tiêu: Hàng là Vị trí, Cột là Vòng thi (1, 2, 3), Giá trị là Điểm trung bình
bang_pivot_diem = pd.pivot_table(
    df_master,
    values='score',     # Tính cái gì? -> Điểm
    index='position',   # Hàng nào? -> Vị trí
    columns='round',    # Cột nào? -> Vòng thi
    aggfunc='mean'      # Tính ntn? -> Trung bình
)

# Làm tròn 2 số lẻ cho đẹp
bang_pivot_diem = bang_pivot_diem.round(2)

print("\n--- 1. Bảng Pivot: Điểm trung bình theo Vị trí & Vòng thi ---")
print(bang_pivot_diem)


# === YÊU CẦU 2: Pivot Mức lương (Vị trí x Kinh nghiệm) ===
# Mục tiêu: So sánh lương của các vị trí theo thâm niên
bang_pivot_luong = pd.pivot_table(
    df_master,
    values='offer_salary',       # Tính cái gì? -> Lương
    index='position',            # Hàng nào? -> Vị trí
    columns='experience_years',  # Cột nào? -> Số năm kinh nghiệm
    aggfunc='mean'
)

print("\n--- 2. Bảng Pivot: Lương trung bình theo Kinh nghiệm ---")
# Định dạng số tiền kiểu 15,000,000 VNĐ cho dễ nhìn (chỉ hiển thị, không đổi dữ liệu gốc)
print(bang_pivot_luong.applymap(lambda x: "{:,.0f} VNĐ".format(x) if pd.notnull(x) else "-"))

# === YÊU CẦU 3: Stack / Unstack (Chuyển đổi dữ liệu) ===
# Stack: Chuyển bảng Pivot (dạng bảng ngang) thành dạng danh sách (dạng dọc)
# Giúp máy tính dễ xử lý hơn khi cần đưa vào database
bang_stack = bang_pivot_diem.stack().reset_index()

# Đổi tên cột cho dễ hiểu
bang_stack.columns = ['Vị trí', 'Vòng thi', 'Điểm Trung Bình']

print("\n--- 3. Demo Stack (Chuyển bảng ngang thành dọc) ---")
print(bang_stack.head())


# === YÊU CẦU 4: Nhận xét nhóm ứng viên mạnh nhất ===
print("\n--- 4. Nhận xét sơ bộ ---")
# Tìm điểm số lớn nhất trong toàn bộ bảng pivot
diem_cao_nhat = bang_pivot_diem.max().max()

print(f"Dựa vào bảng Pivot điểm số, ta thấy:")
print(f"- Điểm trung bình cao nhất đạt được ở một vòng thi là: {diem_cao_nhat}")
print("- Nhìn chung, các vị trí có điểm số khá đồng đều ở mức 6.0 đến 7.5.")


# --- BƯỚC 3: XUẤT FILE KẾT QUẢ ---
bang_pivot_diem.to_csv('ket_qua_pivot_diem.csv', encoding='utf-8-sig')
bang_pivot_luong.to_csv('ket_qua_pivot_luong.csv', encoding='utf-8-sig')



--- 1. Bảng Pivot: Điểm trung bình theo Vị trí & Vòng thi ---
round                1     2     3
position                          
Data Analyst      7.40  6.50  7.56
Python Developer  7.44  6.67  7.10

--- 2. Bảng Pivot: Lương trung bình theo Kinh nghiệm ---
experience_years             1.0             2.0             3.0  \
position                                                           
Data Analyst      20,833,333 VNĐ               -  15,000,000 VNĐ   
Python Developer  20,000,000 VNĐ  15,000,000 VNĐ  15,000,000 VNĐ   

experience_years             4.0             5.0  
position                                          
Data Analyst      20,000,000 VNĐ  20,000,000 VNĐ  
Python Developer  20,000,000 VNĐ               -  

--- 3. Demo Stack (Chuyển bảng ngang thành dọc) ---
             Vị trí  Vòng thi  Điểm Trung Bình
0      Data Analyst         1             7.40
1      Data Analyst         2             6.50
2      Data Analyst         3             7.56
3  Python Developer  

C:\Users\minhh\AppData\Local\Temp\ipykernel_14008\4129943910.py:58: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(bang_pivot_luong.applymap(lambda x: "{:,.0f} VNĐ".format(x) if pd.notnull(x) else "-"))
